In [ ]:
%run IMPORT.ipynb

In [ ]:
%config InlineBackend.figure_format = 'retina'
%matplotlib notebook

### Download data

In [ ]:
%%bash

export ASPERA_SCP_PASS='';
username='';
server=''
ascp -k 3 --ignore-host-key -QTl 100m ${username}@${server}:dbox_content .

parallel_downloads=10
grep 'ukb_l2r\|ukb_cal\|ukb_baf\|ukb_imp\|ukb_hla' dbox_content | grep -v 'bgi' | xargs -i --max-procs=$parallel_downloads bash -c \
"ascp --ignore-host-key -k 1 -QTl 100m ${username}@${server}:{} ./{}"

email=''
pass=''
dkey=''
java -jar EgaDemoClient.jar -p $email $pass -dc *.cip -dck $dkey

mkdir /mnt/data/ukbdata
cd /mnt/data/ukbdata

wget http://biobank.ndph.ox.ac.uk/showcase/auxdata/ukb_snp_qc.txt
wget http://biobank.ndph.ox.ac.uk/showcase/auxdata/ukb_snp_bim.tar
wget http://biobank.ndph.ox.ac.uk/showcase/auxdata/ukb_imp_bgi.tgz
wget http://biobank.ndph.ox.ac.uk/showcase/auxdata/ukb_imp_mfi.tgz

ukbgene rel -a.ukbkey

In [ ]:
%%bash

for chr in {1..22}; do
    gzip -d ukb_bgi_chr${chr}_v2.bgi.gz;
    gzip -d ukb_imp_chr${chr}_v2.bgen.gz; done

rename 's/_bgi_/_imp_/g' ukb_bgi_*.bgi
rename 's/.bgi/.bgen.bgi/g' ukb_imp_*.bgi
for chr in {1..22}; do cp ukb_imp_chr$chr.bgi ukb_imp_chr$chr.bgi_original; done

### Generate 10k reference

In [ ]:
for chr in {01..22}; do
    cat-bgen -g 500k_mac200_info07/chr${chr}_*bgen -og merged_chromosomes/chr$chr.bgen;
    plink2 --bgen merged_chromosomes/chr$chr.bgen --sample ukbxxxx_imp_chr1_v2_sxxxxxx.sample --make-pgen --out merged_chromosomes/chr$chr;
    rm merged_chromosomes/chr$chr.bgen; done
    plink2 --pfile merged_chromosomes/chr$chr --keep 10k_reference.index --make-bed --out 10kref/chr$chr;
    done

In [ ]:
for f in {01..22}; do
plink2 --pfile merged/chr$f --keep 10k_reference.index --make-bed --memory 80000 --out 10kref_v3/chr$f;
done

In [ ]:
slice300k=300k_df_slice.index

parallel --eta -j 1 plink2 --pfile merged_chromosomes/{/.} --extract {} --keep $slice300k --export A-transpose --out {.} ::: w_hm3/*snps